## [SQLAlchemy チュートリアル概要｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/bee9e4)

In [2]:
import sqlalchemy
sqlalchemy.__version__

'2.0.39'

## [接続の確立 - エンジンの作成｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/754e70)

In [3]:
from sqlalchemy import create_engine

# SQLiteのインメモリデータベースに接続するEngineを作成
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

## [トランザクションとDBAPIの操作｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/f91303)

In [4]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

2025-03-12 14:37:03,892 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:37:03,894 INFO sqlalchemy.engine.Engine select 'hello world'
2025-03-12 14:37:03,895 INFO sqlalchemy.engine.Engine [generated in 0.00218s] ()
[('hello world',)]
2025-03-12 14:37:03,896 INFO sqlalchemy.engine.Engine ROLLBACK


In [5]:
with engine.connect() as conn:
    conn.execute(text("CREATE TABLE some_table (x int, y int)"))
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 1, "y": 1}, {"x": 2, "y": 4}]
    )
    conn.commit()

2025-03-12 14:38:26,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:38:26,717 INFO sqlalchemy.engine.Engine CREATE TABLE some_table (x int, y int)
2025-03-12 14:38:26,718 INFO sqlalchemy.engine.Engine [generated in 0.00154s] ()
2025-03-12 14:38:26,719 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-03-12 14:38:26,720 INFO sqlalchemy.engine.Engine [generated in 0.00065s] [(1, 1), (2, 4)]
2025-03-12 14:38:26,720 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
with engine.begin() as conn:
    conn.execute(
        text("INSERT INTO some_table (x, y) VALUES (:x, :y)"),
        [{"x": 6, "y": 8}, {"x": 9, "y": 10}]
    )

2025-03-12 14:39:06,212 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:39:06,213 INFO sqlalchemy.engine.Engine INSERT INTO some_table (x, y) VALUES (?, ?)
2025-03-12 14:39:06,214 INFO sqlalchemy.engine.Engine [cached since 39.49s ago] [(6, 8), (9, 10)]
2025-03-12 14:39:06,215 INFO sqlalchemy.engine.Engine COMMIT


In [8]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table"))
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-03-12 14:39:42,608 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:39:42,609 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table
2025-03-12 14:39:42,610 INFO sqlalchemy.engine.Engine [cached since 14.71s ago] ()
x: 1, y: 1
x: 2, y: 4
x: 6, y: 8
x: 9, y: 10
2025-03-12 14:39:42,611 INFO sqlalchemy.engine.Engine COMMIT


In [12]:
with engine.begin() as conn:
    result = conn.execute(text("SELECT x, y FROM some_table WHERE y > :y"), {"y": 5})

    for row in result:
        print(f"x: {row.x}, y: {row.y}")

print(type(result))

2025-03-12 14:41:24,658 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:41:24,659 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ?
2025-03-12 14:41:24,660 INFO sqlalchemy.engine.Engine [cached since 44.88s ago] (5,)
x: 6, y: 8
x: 9, y: 10
2025-03-12 14:41:24,661 INFO sqlalchemy.engine.Engine COMMIT
<class 'sqlalchemy.engine.cursor.CursorResult'>


In [13]:
from sqlalchemy.orm import Session

stmt = text("SELECT x, y FROM some_table WHERE y > :y ORDER BY x, y")
with Session(engine) as session:
    result = session.execute(stmt, {"y": 5})
    for row in result:
        print(f"x: {row.x}, y: {row.y}")

2025-03-12 14:42:53,744 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:42:53,745 INFO sqlalchemy.engine.Engine SELECT x, y FROM some_table WHERE y > ? ORDER BY x, y
2025-03-12 14:42:53,746 INFO sqlalchemy.engine.Engine [generated in 0.00137s] (5,)
x: 6, y: 8
x: 9, y: 10
2025-03-12 14:42:53,748 INFO sqlalchemy.engine.Engine ROLLBACK


## [データベースメタデータの操作｜SQLAlchemy 2.0 チュートリアル](https://zenn.dev/jin1125/books/e91183cdff5f2a/viewer/3251b3)

In [14]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [15]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String)
)

In [16]:
print(user_table.c.name)
print(user_table.c.keys())

user_account.name
['id', 'name', 'fullname']


In [18]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False)
)

In [19]:
metadata_obj.create_all(engine)

2025-03-12 14:47:16,313 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:47:16,314 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user_account")
2025-03-12 14:47:16,315 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:47:16,316 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user_account")
2025-03-12 14:47:16,317 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:47:16,318 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2025-03-12 14:47:16,319 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:47:16,320 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2025-03-12 14:47:16,321 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:47:16,323 INFO sqlalchemy.engine.Engine 
CREATE TABLE user_account (
	id INTEGER NOT NULL, 
	name VARCHAR(30), 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2025-03-12 14:47:16,324 INFO sqlalchemy.engine.Engine [no key 0.00075s] ()
2025-03-12 14:47:16,326 INFO sqlalchemy.engine.Engine 
C

In [20]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [21]:
from sqlalchemy.orm import Mapped, mapped_column, relationship
from typing import List, Optional


# Userクラスの定義
class User(Base):
    __tablename__ = "user_account"

    id: Mapped[int] = mapped_column(primary_key=True)
    name: Mapped[str] = mapped_column(String(30))
    fullname: Mapped[Optional[str]]

    addresses: Mapped[List["Address"]] = relationship(back_populates="user")


# Addressクラスの定義
class Address(Base):
    __tablename__ = "address"

    id: Mapped[int] = mapped_column(primary_key=True)
    email_address: Mapped[str]
    user_id: Mapped[int] = mapped_column(ForeignKey("user_account.id"))

    user: Mapped[User] = relationship(back_populates="addresses")

In [22]:
some_table = Table("some_table", metadata_obj, autoload_with=engine)

2025-03-12 14:49:31,465 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-12 14:49:31,466 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("some_table")
2025-03-12 14:49:31,466 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:49:31,467 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type in ('table', 'view')
2025-03-12 14:49:31,469 INFO sqlalchemy.engine.Engine [raw sql] ('some_table',)
2025-03-12 14:49:31,471 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("some_table")
2025-03-12 14:49:31,473 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:49:31,473 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("some_table")
2025-03-12 14:49:31,474 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-03-12 14:49:31,474 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type i

In [23]:
print(some_table.c.keys())

['x', 'y']
